#### 1. Create a model that includes only age, fare, and pclass. Does this model perform better than your baseline?

#### 2. Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

#### 3. Try out other combinations of features and models.

#### 4. Use you best 3 models to predict and evaluate on your validate sample.

#### 5. Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?

#### **Bonus1** How do different strategies for handling the missing values in the age column affect model performance?

#### **Bonus2** How do different strategies for encoding sex affect model performance?

#### **Bonus3** scikit-learn's LogisticRegression classifier is actually applying a regularization penalty to the coefficients by default. This penalty causes the magnitude of the coefficients in the resulting model to be smaller than they otherwise would be. This value can be modified with the C hyper parameter. Small values of C correspond to a larger penalty, and large values of C correspond to a smaller penalty.
Try out the following values for C and note how the coefficients and the model's performance on both the dataset it was trained on and on the validate split are affected.
C
=
.01
,
.1
,
1
,
10
,
100
,
1000

#### **Bonus Bonus** how does scaling the data interact with your choice of C?